In [ ]:
# stdlib
import pathlib

# Environment
from flappy_bird import FlappyBirdEnvV0
import gym
from gym.wrappers import GrayScaleObservation, ResizeObservation
from proximal_policy_optimization import FrameSkip

# Jax, Flax, Optax
import jax
import jax.numpy as jnp
from flax.metrics import tensorboard
from flax.training import checkpoints
import optax

# PPO
import proximal_policy_optimization as ppo

# other
from tqdm.notebook import tqdm


In [ ]:
%env XLA_PYTHON_CLIENT_MEM_FRACTION=0.85

Environment creation

In [4]:
# Not really needed, but allows us to use 'gym.make'.
gym.register(
    id="FlappyBird-v0",
    entry_point="flappy_bird.envs:FlappyBirdEnvV0",
)

def make_env():
    env = gym.make("FlappyBird-v0")
    env = ResizeObservation(env, (84, 84))
    env = GrayScaleObservation(env, keep_dim=True)
    env = FrameSkip(env)
    return env


def make_vector_env(num_envs: int = 8, asynchronous: bool = False):
    if asynchronous:
        env = gym.vector.AsyncVectorEnv([make_env for _ in range(num_envs)])  # type: ignore
    else:
        env = gym.vector.SyncVectorEnv([make_env for _ in range(num_envs)])  # type: ignore
    return env


train_env = make_vector_env()
eval_env = make_env()


Configuration

In [5]:
def ppo_num_opt_steps(
    total_frames: int, horizon: int, n_actors: int, epochs: int, mini_batch_size: int
) -> int:
    """Compute the number of optimization steps."""
    batch_size = horizon * n_actors
    # Number of frames we see per train step
    frames_per_train_step = batch_size
    # Number of times we call optimizer per step
    opt_steps_per_train_step = epochs * (batch_size // mini_batch_size)
    # Number of train steps
    num_train_steps = total_frames // frames_per_train_step
    # Total number of optimizer calls
    total_opt_steps = opt_steps_per_train_step * num_train_steps

    return total_opt_steps

# These parameters are from https://github.com/araffin/rl-baselines-zoo/blob/master/hyperparams/ppo2.yml 
total_frames = int(1e7)
n_actors = 8
horizon = 128
mini_batch_size = 256
epochs = 4
total_opt_steps = ppo_num_opt_steps(
    total_frames, horizon, n_actors, epochs, mini_batch_size
)

gamma = 0.99
lam = 0.95
epsilon = optax.linear_schedule(0.1, 0.0, total_opt_steps)
c1 = 0.5
c2 = 0.01
learning_rate = optax.linear_schedule(2.5e-4, 0.0, total_opt_steps)
max_grad_norm = 0.5
clip_reward = False

# Configuration
config = ppo.PPOConfig(
    n_actors=n_actors,
    total_frames=total_frames,
    horizon=horizon,
    mini_batch_size=mini_batch_size,
    lam=lam,
    gamma=gamma,
    epochs=epochs,
    c1=c1,
    c2=c2,
    epsilon=epsilon,
    clip_reward=clip_reward,
)


Create the train state

In [6]:
# Create Model
key = jax.random.PRNGKey(0)
n_hidden = 512
n_actions = train_env.action_space[0].n  # type: ignore
model = ppo.ActorCriticCnn(n_hidden=n_hidden, n_actions=n_actions)

# Initialize model
observation = ppo.env_reset(train_env)
key, rng = jax.random.split(key, 2)
params = model.init(rng, observation)
state = ppo.PPOTrainState.create(
    apply_fn=model.apply,
    params=params,
    lr=learning_rate,
    config=config,
    max_grad_norm=max_grad_norm,
)
del params


Configure logging and checkpointing 

In [7]:
checkpoint_dir = (
    pathlib.Path(".").absolute().joinpath("checkpoints/flappy_bird/run1").as_posix()
)
log_dir = pathlib.Path(".").absolute().joinpath("logs/flappy_bird/run1").as_posix()

summary_writer = tensorboard.SummaryWriter(log_dir)
summary_writer.hparams(config._asdict())

log_frequency = 1
eval_frequency = 25
eval_episodes = 25

Train!

In [9]:
batch_size = config.horizon * config.n_actors
frames_per_train_step = batch_size
num_train_steps = config.total_frames // frames_per_train_step

reward = 0.0

horizon = state.config.horizon
gamma = state.config.gamma
lam = state.config.lam

with tqdm(range(num_train_steps)) as t:
    for step in t:
        frame = step * frames_per_train_step
        t.set_description(f"frame: {step}")

        key, rng1, rng2 = jax.random.split(key, 3)
        trajectory, observation = ppo.create_trajectory(
            observation,
            state.apply_fn,
            state.params,
            train_env,
            rng1,
            horizon,
            gamma,
            lam,
        )
        state, losses = ppo.train_step(state, trajectory, rng2)

        if step % log_frequency == 0:
            summary_writer.scalar("train/loss", losses["total"], frame)
            summary_writer.scalar("train/loss-actor", losses["actor"], frame)
            summary_writer.scalar("train/loss-critic", losses["critic"], frame)
            summary_writer.scalar("train/loss-entropy", losses["entropy"], frame)
            summary_writer.scalar("train/learning-rate", state.learning_rate(), frame)
            summary_writer.scalar("train/clipping", state.epsilon(), frame)

        if step % eval_frequency == 0:
            key, rng = jax.random.split(key, 2)
            reward = ppo.evaluate_model(state, eval_env, eval_episodes, rng)
            summary_writer.scalar("train/reward", reward, frame)

        t.set_description_str(f"loss: {losses['total']}, reward: {reward}")

        if checkpoint_dir is not None:
            checkpoints.save_checkpoint(checkpoint_dir, state, frame)


  0%|          | 0/9765 [00:00<?, ?it/s]